In [2]:
%matplotlib inline
%load_ext autoreload

%autoreload 2

import pandas as pd
import numpy as np
import pyDOE

from pandas_helper import *


from generate_doe import generate_doe

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def cartesian(df1, df2):
    rows = itertools.product(df1.iterrows(), df2.iterrows())

    df = pd.DataFrame(left.append(right) for (_, left), (_, right) in rows)
    return df.reset_index(drop=True)

# number of simulations we can afford

In [4]:
n_sample=100

# weighted case

In [5]:
## dimensions we want to cover

In [6]:
doe_params_simple  = pd.read_excel("doe_parameters_weighted.xlsx", sheet_name="simple_dimensions")
doe_params_simple

,demand,OPEC
0,low_1.5,opec_full
1,med_2,opec_contrained
2,high_3,NaN


In [69]:
doe_params_coump  = pd.read_excel("doe_parameters_weighted.xlsx", sheet_name="coumpounded_dimensions")
doe_params_coump.columns.name = "dim"
doe_params_coump.index.name = "case"
doe_params_coump

dim,fiscal_pressure,fiscal_regime
case,,
0,fiscal_low,fi_A
1,fiscal_high,fi_B
2,NaN,fi_C


In [75]:
country_weight = pd.read_excel("doe_parameters_weighted.xlsx", sheet_name="country_weight").set_index("country").squeeze()
country_weight.index.name = "country"
country_weight


country
Brazil       5
Colombia     5
Venezuela    5
trinindad    1
uruguay      1
Name: weight, dtype: int64

In [76]:
coumpounded_dims =broadcast_simple(doe_params_coump.stack(), country_weight.index).unstack().unstack()

coumpounded_dims

country          Brazil                      Colombia                \
dim     fiscal_pressure fiscal_regime fiscal_pressure fiscal_regime   
case                                                                  
0            fiscal_low          fi_A      fiscal_low          fi_A   
1           fiscal_high          fi_B     fiscal_high          fi_B   
2                   NaN          fi_C             NaN          fi_C   

country       Venezuela                     trinindad                \
dim     fiscal_pressure fiscal_regime fiscal_pressure fiscal_regime   
case                                                                  
0            fiscal_low          fi_A      fiscal_low          fi_A   
1           fiscal_high          fi_B     fiscal_high          fi_B   
2                   NaN          fi_C             NaN          fi_C   

country         uruguay                
dim     fiscal_pressure fiscal_regime  
case                                   
0            fiscal_low          fi_A  
1           fiscal_high          fi_B  
2                   NaN          fi_C

## merge column indexes

In [90]:
def merge_column_index(coumpounded_dims):
    hop =coumpounded_dims.stack(["dim","country"]).reset_index()
    hop["dim_country"] = hop.dim+"_"+hop.country
    doe_params_developped = pd.concat([doe_params_simple, 
                                             hop.set_index(["dim_country","case" ])[0].unstack("dim_country") ],
                                            axis=1)
    return doe_params_developped

merge_column_index(coumpounded_dims)

,demand,OPEC,fiscal_pressure_Brazil,fiscal_pressure_Colombia,fiscal_pressure_Venezuela,fiscal_pressure_trinindad,fiscal_pressure_uruguay,fiscal_regime_Brazil,fiscal_regime_Colombia,fiscal_regime_Venezuela,fiscal_regime_trinindad,fiscal_regime_uruguay
0,low_1.5,opec_full,fiscal_low,fiscal_low,fiscal_low,fiscal_low,fiscal_low,fi_A,fi_A,fi_A,fi_A,fi_A
1,med_2,opec_contrained,fiscal_high,fiscal_high,fiscal_high,fiscal_high,fiscal_high,fi_B,fi_B,fi_B,fi_B,fi_B
2,high_3,NaN,NaN,NaN,NaN,NaN,NaN,fi_C,fi_C,fi_C,fi_C,fi_C


# duplicate accoridng to country weight

In [116]:
df = coumpounded_dims.stack().stack().swaplevel("case","country").reset_index().rename(columns={0:"value"})
df["weight"] = np.nan

df_long = df.copy()

for c in df.country.unique():
    for i in np.arange(country_weight[c]) :
        df_long = df_long.append(df.query("country=='"+c+"'").assign(weight=i))
df_long = df_long.dropna() 
df_long

,country,dim,case,value,weight
0,Brazil,fiscal_pressure,0,fiscal_low,0.0
5,Brazil,fiscal_regime,0,fi_A,0.0
10,Brazil,fiscal_pressure,1,fiscal_high,0.0
15,Brazil,fiscal_regime,1,fi_B,0.0
20,Brazil,fiscal_regime,2,fi_C,0.0
0,Brazil,fiscal_pressure,0,fiscal_low,1.0
5,Brazil,fiscal_regime,0,fi_A,1.0
10,Brazil,fiscal_pressure,1,fiscal_high,1.0
15,Brazil,fiscal_regime,1,fi_B,1.0
20,Brazil,fiscal_regime,2,fi_C,1.0


In [117]:
coumpounded_dims

country          Brazil                      Colombia                \
dim     fiscal_pressure fiscal_regime fiscal_pressure fiscal_regime   
case                                                                  
0            fiscal_low          fi_A      fiscal_low          fi_A   
1           fiscal_high          fi_B     fiscal_high          fi_B   
2                   NaN          fi_C             NaN          fi_C   

country       Venezuela                     trinindad                \
dim     fiscal_pressure fiscal_regime fiscal_pressure fiscal_regime   
case                                                                  
0            fiscal_low          fi_A      fiscal_low          fi_A   
1           fiscal_high          fi_B     fiscal_high          fi_B   
2                   NaN          fi_C             NaN          fi_C   

country         uruguay                
dim     fiscal_pressure fiscal_regime  
case                                   
0            fiscal_low          fi_A  
1           fiscal_high          fi_B  
2                   NaN          fi_C

In [181]:
a = df_long.set_index(["dim", "country", "case", "weight"])["value"].unstack("country").unstack("dim").reset_index("weight")
w = a.pop("weight")
w
#for c in a:
#    pass
a=a.apply(lambda s:s+'_w'+w.astype(str)).reset_index("case", drop=True)
a.index.name = "case"
a= merge_column_index(a)


In [190]:
mydoe_extended =generate_doe(n_sample*100,a , iterations=1)
mydoe_extended.head(20)

new candidate solution found with max,abs corrcoef = 0.9804664558302972


,demand,OPEC,fiscal_pressure_Brazil,fiscal_pressure_Colombia,fiscal_pressure_Venezuela,fiscal_pressure_trinindad,fiscal_pressure_uruguay,fiscal_regime_Brazil,fiscal_regime_Colombia,fiscal_regime_Venezuela,fiscal_regime_trinindad,fiscal_regime_uruguay
0,high_3,opec_contrained,fiscal_low_w0.0,fiscal_high_w3.0,fiscal_high_w4.0,fiscal_low_w0.0,fiscal_high_w0.0,fi_A_w0.0,fi_B_w3.0,fi_B_w4.0,fi_B_w0.0,fi_A_w0.0
1,med_2,opec_contrained,fiscal_high_w3.0,fiscal_low_w2.0,fiscal_low_w4.0,fiscal_high_w0.0,fiscal_high_w0.0,fi_A_w3.0,fi_A_w0.0,fi_B_w2.0,fi_A_w0.0,fi_A_w0.0
2,med_2,opec_full,fiscal_high_w2.0,fiscal_high_w3.0,fiscal_high_w0.0,fiscal_high_w0.0,fiscal_low_w0.0,fi_C_w0.0,fi_A_w2.0,fi_C_w1.0,fi_C_w0.0,fi_C_w0.0
3,med_2,opec_contrained,fiscal_high_w3.0,fiscal_low_w1.0,fiscal_low_w0.0,fiscal_low_w0.0,fiscal_low_w0.0,fi_B_w1.0,fi_C_w1.0,fi_A_w3.0,fi_A_w0.0,fi_C_w0.0
4,high_3,opec_contrained,fiscal_high_w1.0,fiscal_low_w4.0,fiscal_low_w3.0,fiscal_high_w0.0,fiscal_low_w0.0,fi_A_w1.0,fi_A_w3.0,fi_C_w2.0,fi_B_w0.0,fi_B_w0.0
5,low_1.5,opec_full,fiscal_high_w3.0,fiscal_high_w1.0,fiscal_high_w4.0,fiscal_high_w0.0,fiscal_low_w0.0,fi_A_w2.0,fi_C_w3.0,fi_A_w0.0,fi_A_w0.0,fi_B_w0.0
6,low_1.5,opec_full,fiscal_high_w3.0,fiscal_low_w0.0,fiscal_low_w0.0,fiscal_high_w0.0,fiscal_low_w0.0,fi_C_w2.0,fi_B_w2.0,fi_A_w0.0,fi_C_w0.0,fi_B_w0.0
7,low_1.5,opec_full,fiscal_high_w3.0,fiscal_high_w1.0,fiscal_low_w2.0,fiscal_high_w0.0,fiscal_high_w0.0,fi_C_w2.0,fi_C_w0.0,fi_C_w4.0,fi_C_w0.0,fi_C_w0.0
8,high_3,opec_full,fiscal_low_w0.0,fiscal_high_w3.0,fiscal_high_w2.0,fiscal_low_w0.0,fiscal_low_w0.0,fi_A_w2.0,fi_B_w4.0,fi_C_w3.0,fi_B_w0.0,fi_C_w0.0
9,med_2,opec_full,fiscal_low_w0.0,fiscal_high_w4.0,fiscal_high_w0.0,fiscal_low_w0.0,fiscal_low_w0.0,fi_A_w1.0,fi_B_w4.0,fi_B_w1.0,fi_C_w0.0,fi_C_w0.0


In [192]:
mydoe_extended

,demand,OPEC,fiscal_pressure_Brazil,fiscal_pressure_Colombia,fiscal_pressure_Venezuela,fiscal_pressure_trinindad,fiscal_pressure_uruguay,fiscal_regime_Brazil,fiscal_regime_Colombia,fiscal_regime_Venezuela,fiscal_regime_trinindad,fiscal_regime_uruguay
0,high_3,opec_contrained,fiscal_low_w0.0,fiscal_high_w3.0,fiscal_high_w4.0,fiscal_low_w0.0,fiscal_high_w0.0,fi_A_w0.0,fi_B_w3.0,fi_B_w4.0,fi_B_w0.0,fi_A_w0.0
1,med_2,opec_contrained,fiscal_high_w3.0,fiscal_low_w2.0,fiscal_low_w4.0,fiscal_high_w0.0,fiscal_high_w0.0,fi_A_w3.0,fi_A_w0.0,fi_B_w2.0,fi_A_w0.0,fi_A_w0.0
2,med_2,opec_full,fiscal_high_w2.0,fiscal_high_w3.0,fiscal_high_w0.0,fiscal_high_w0.0,fiscal_low_w0.0,fi_C_w0.0,fi_A_w2.0,fi_C_w1.0,fi_C_w0.0,fi_C_w0.0
3,med_2,opec_contrained,fiscal_high_w3.0,fiscal_low_w1.0,fiscal_low_w0.0,fiscal_low_w0.0,fiscal_low_w0.0,fi_B_w1.0,fi_C_w1.0,fi_A_w3.0,fi_A_w0.0,fi_C_w0.0
4,high_3,opec_contrained,fiscal_high_w1.0,fiscal_low_w4.0,fiscal_low_w3.0,fiscal_high_w0.0,fiscal_low_w0.0,fi_A_w1.0,fi_A_w3.0,fi_C_w2.0,fi_B_w0.0,fi_B_w0.0
5,low_1.5,opec_full,fiscal_high_w3.0,fiscal_high_w1.0,fiscal_high_w4.0,fiscal_high_w0.0,fiscal_low_w0.0,fi_A_w2.0,fi_C_w3.0,fi_A_w0.0,fi_A_w0.0,fi_B_w0.0
6,low_1.5,opec_full,fiscal_high_w3.0,fiscal_low_w0.0,fiscal_low_w0.0,fiscal_high_w0.0,fiscal_low_w0.0,fi_C_w2.0,fi_B_w2.0,fi_A_w0.0,fi_C_w0.0,fi_B_w0.0
7,low_1.5,opec_full,fiscal_high_w3.0,fiscal_high_w1.0,fiscal_low_w2.0,fiscal_high_w0.0,fiscal_high_w0.0,fi_C_w2.0,fi_C_w0.0,fi_C_w4.0,fi_C_w0.0,fi_C_w0.0
8,high_3,opec_full,fiscal_low_w0.0,fiscal_high_w3.0,fiscal_high_w2.0,fiscal_low_w0.0,fiscal_low_w0.0,fi_A_w2.0,fi_B_w4.0,fi_C_w3.0,fi_B_w0.0,fi_C_w0.0
9,med_2,opec_full,fiscal_low_w0.0,fiscal_high_w4.0,fiscal_high_w0.0,fiscal_low_w0.0,fiscal_low_w0.0,fi_A_w1.0,fi_B_w4.0,fi_B_w1.0,fi_C_w0.0,fi_C_w0.0


In [191]:
mydoe_extended.applymap(lambda s: s.split("_w")[0]).drop_duplicates()

,demand,OPEC,fiscal_pressure_Brazil,fiscal_pressure_Colombia,fiscal_pressure_Venezuela,fiscal_pressure_trinindad,fiscal_pressure_uruguay,fiscal_regime_Brazil,fiscal_regime_Colombia,fiscal_regime_Venezuela,fiscal_regime_trinindad,fiscal_regime_uruguay
0,high_3,opec_contrained,fiscal_low,fiscal_high,fiscal_high,fiscal_low,fiscal_high,fi_A,fi_B,fi_B,fi_B,fi_A
1,med_2,opec_contrained,fiscal_high,fiscal_low,fiscal_low,fiscal_high,fiscal_high,fi_A,fi_A,fi_B,fi_A,fi_A
2,med_2,opec_full,fiscal_high,fiscal_high,fiscal_high,fiscal_high,fiscal_low,fi_C,fi_A,fi_C,fi_C,fi_C
3,med_2,opec_contrained,fiscal_high,fiscal_low,fiscal_low,fiscal_low,fiscal_low,fi_B,fi_C,fi_A,fi_A,fi_C
4,high_3,opec_contrained,fiscal_high,fiscal_low,fiscal_low,fiscal_high,fiscal_low,fi_A,fi_A,fi_C,fi_B,fi_B
5,low_1.5,opec_full,fiscal_high,fiscal_high,fiscal_high,fiscal_high,fiscal_low,fi_A,fi_C,fi_A,fi_A,fi_B
6,low_1.5,opec_full,fiscal_high,fiscal_low,fiscal_low,fiscal_high,fiscal_low,fi_C,fi_B,fi_A,fi_C,fi_B
7,low_1.5,opec_full,fiscal_high,fiscal_high,fiscal_low,fiscal_high,fiscal_high,fi_C,fi_C,fi_C,fi_C,fi_C
8,high_3,opec_full,fiscal_low,fiscal_high,fiscal_high,fiscal_low,fiscal_low,fi_A,fi_B,fi_C,fi_B,fi_C
9,med_2,opec_full,fiscal_low,fiscal_high,fiscal_high,fiscal_low,fiscal_low,fi_A,fi_B,fi_B,fi_C,fi_C


In [185]:
"aaa_w0.0"

'aaa'

In [22]:
df.apply(pd.value_counts).sum(axis=1).sort_values()

med_2               32.0
high_3              34.0
low_1.5             34.0
opec_contrained     50.0
opec_full           50.0
fi_B               165.0
fi_A               166.0
fi_C               169.0
fiscal_low         248.0
fiscal_high        252.0
dtype: float64

This above shows that the values with 3 possibilities appeared 33% times and the values with 2 appeared 50 %times

In [23]:
df.drop_duplicates()

,demand,OPEC,fiscal_pressure_Brazil,fiscal_pressure_Colombia,fiscal_pressure_Venezuela,fiscal_pressure_trinindad,fiscal_pressure_uruguay,fiscal_regime_Brazil,fiscal_regime_Colombia,fiscal_regime_Venezuela,fiscal_regime_trinindad,fiscal_regime_uruguay
0,med_2,opec_full,fiscal_low,fiscal_low,fiscal_high,fiscal_high,fiscal_high,fi_A,fi_B,fi_B,fi_A,fi_B
1,low_1.5,opec_contrained,fiscal_high,fiscal_low,fiscal_high,fiscal_low,fiscal_high,fi_A,fi_C,fi_A,fi_C,fi_B
2,high_3,opec_full,fiscal_high,fiscal_high,fiscal_low,fiscal_low,fiscal_high,fi_C,fi_A,fi_A,fi_A,fi_B
3,low_1.5,opec_full,fiscal_low,fiscal_low,fiscal_high,fiscal_low,fiscal_high,fi_C,fi_C,fi_A,fi_C,fi_A
4,med_2,opec_contrained,fiscal_low,fiscal_low,fiscal_high,fiscal_high,fiscal_high,fi_B,fi_C,fi_B,fi_C,fi_B
5,low_1.5,opec_contrained,fiscal_low,fiscal_high,fiscal_high,fiscal_high,fiscal_high,fi_C,fi_B,fi_C,fi_C,fi_B
6,high_3,opec_full,fiscal_low,fiscal_high,fiscal_low,fiscal_low,fiscal_low,fi_C,fi_C,fi_C,fi_B,fi_C
7,high_3,opec_full,fiscal_low,fiscal_low,fiscal_low,fiscal_low,fiscal_high,fi_C,fi_C,fi_C,fi_A,fi_B
8,low_1.5,opec_full,fiscal_high,fiscal_high,fiscal_low,fiscal_high,fiscal_high,fi_B,fi_B,fi_B,fi_B,fi_B
9,high_3,opec_full,fiscal_low,fiscal_high,fiscal_high,fiscal_low,fiscal_low,fi_B,fi_B,fi_C,fi_B,fi_A
